# Bibliotecas e preparação do ambiente

## Bibliotecas

In [2]:
import configparser
import json
import os
import pandas as pd
import re
import requests
import string
import uuid


from collections import Counter
from datetime import datetime, timedelta
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pathlib import Path
from unicodedata import normalize
from urllib.parse import urlparse

## Constantes

In [2]:
# credentials file location
CREDS_FILE = r'C:\Users\maxwelfreitas\creds.ini'
# time format for json result files
RESULT_TS_FORMAT = '%Y%m%d%H%M%S%f'
# time format for wordcloud files
OUTPUT_TS_FORMAT = '%d/%m/%Y %H:%M:%S'
# sites to ignore results
BLACK_LISTED_SITES = ['gov.br', 'fccid.io', 'wer-hat-angerufen.info']
#output folders
SEARCH_RESULTS_FOLDER = 'datasets/searchresults'
ANNOTATITIONS_FOLDER = 'datasets/annotations'

In [98]:
with open('blacklist.ini','w') as f:
    f.writelines(BLACK_LISTED_SITES)

## Funções auxiliares

### Uso geral

In [3]:
def parse_site(url):
    return '.'.join(urlparse(url).netloc.split('.')[-3:])

In [4]:
def is_black_listed_site(url,black_listed_sites=BLACK_LISTED_SITES):
    for site in black_listed_sites:
        if site in url:
            return True
    return False

In [5]:
def tokenizer(doc,normalize_words=False):

    stop_words = stopwords.words('portuguese')
    stop_words.extend(stopwords.words('english'))
    stop_words.extend(list(string.punctuation))

    # stopwords específicas do domínio
    stop_words.extend(['cm', 'feature', 'features', 'informações', 'itens', 'leve', 'list', 'nulo', 'package', 
                       'pacote', 'pacotes', 'recurso', 'tamanho', 'ver', 'anatel', 'laranja', '.', '...',
                       'complementares', 'peça'])

    if normalize_words:
        doc = doc.lower()
        doc = normalize('NFKD', doc).encode('ASCII', 'ignore').decode('ASCII')

    # CountVectorizer token pattern
    pattern = r'\b\w\w+\b'
    tokens = [token for token in re.findall(pattern,doc) if token.lower() not in stop_words]
    # remove number-only tokens
    tokens = [token for token in tokens if not re.match('\d+',token)]

    # NLTK tokenizer
    # tokens = [token for token in word_tokenize(doc) if token.lower() not in stop_words]

    return tokens


In [6]:
# file should be a Path object
def parse_result_file(file, nWords=25):

    search_date, search_engine, search_term, _ = re.split('[_.]',file.name)
    search_site = None
    
    with open(file) as f:
        results = json.load(f)
        lines = []
        if search_engine == 'GOOGLE':
            # results without items in keys are empty
            if 'items' in results.keys():
                for item in results['items']:
                    search_site = item['displayLink']
                    if is_black_listed_site(search_site):
                        continue
                    else:
                        search_site = parse_site(search_site)
                        lines.append(item['title'])
                        if 'snippet' in item.keys():
                            lines.append(item['snippet'])
                        
        elif search_engine == 'BING':
            # results without webPages in keys are empty
            if 'webPages' in results.keys():
                for item in results['webPages']['value'][:10]:
                    search_site = item['url']
                    if is_black_listed_site(search_site):
                        continue
                    else:
                        search_site = parse_site(search_site)
                        lines.append(item['name'])
                        if 'snippet' in item.keys():
                            lines.append(item['snippet'])
                    
        if len(lines) >= 1:
            words = tokenizer(' '.join(lines))
            words_conter = Counter(words)
            wordCloud_dict = {key:value for key,value in words_conter.most_common(nWords)}
            wordCloud_json = json.dumps(wordCloud_dict, ensure_ascii=False)
        else:
            wordCloud_json = ''
        
        wourdCloudInfo_dict = {
            'metaData': {
                'Version': 1,
                'Source': search_engine,
                'Mode': 'API',
                'Fields': ['Name', 'Snippet'],
                'nWords': nWords
            },
            'searchedWord': search_term,
            'cloudOfWords': wordCloud_json
        }
    
        wourdCloudInfo_json = json.dumps(wourdCloudInfo_dict, ensure_ascii=False)
    
        return {'ID': str(uuid.uuid4()),
                'DataHora': datetime.strptime(search_date,RESULT_TS_FORMAT).strftime(OUTPUT_TS_FORMAT),
                'Computador': os.environ['COMPUTERNAME'],
                'Usuário': 'E!', 
                'Homologação': f'{search_term[:5]}-{search_term[5:7]}-{search_term[-5:]}', 
                'Atributo': 'WordCloud',
                'Valor': wourdCloudInfo_json,               
                'Buscadora': search_engine}

### Pesquisa

In [7]:
def load_creds(creds_file=None):

    if creds_file == None:
        creds_file = Path(os.environ['USERPROFILE'],'creds.ini')
        
    if not os.path.exists(creds_file):
        raise FileNotFoundError('Credentials file not found', creds_file)
    
    creds = configparser.ConfigParser()
    creds.read(creds_file)
    
    bing_search_api_key = creds['BING_SEARCH']['bing_search_api_key']
    bing_search_endpoint = creds['BING_SEARCH']['bing_search_endpoint']
    google_search_api_key = creds['GOOGLE_SEARCH']['google_search_api_key']
    google_search_engine_id = creds['GOOGLE_SEARCH']['google_search_engine_id']
    google_search_endpoint = creds['GOOGLE_SEARCH']['google_search_endpoint']
    
    creds = {'BING': {'bing_search_api_key': bing_search_api_key, 
                      'bing_search_endpoint': bing_search_endpoint },
             'GOOGLE': {'google_search_api_key': google_search_api_key, 
                        'google_search_engine_id': google_search_engine_id, 
                        'google_search_endpoint': google_search_endpoint }}
    return creds

In [8]:
def save_results_file(search_results):
    if 'kind' in search_results.keys(): # Google
        search_engine = 'GOOGLE'
        search_term = search_results['queries']['request'][0]['searchTerms']
    elif '_type' in search_results.keys(): # Bing
        search_engine = 'BING'
        search_term = search_results['queryContext']['originalQuery']
    else:
        return None
        
    result_ts = datetime.now().strftime(RESULT_TS_FORMAT)
    results_filename = f'{result_ts}_{search_engine}_{search_term}.json'
    file_to_save = Path(SEARCH_RESULTS_FOLDER,results_filename)
    
    with open(file_to_save, 'w') as f:
        json.dump(search_results,f, indent=2)    

    return file_to_save

In [10]:
def google_search(search_term,creds):
    # load credentials
    google_search_api_key = creds['GOOGLE']['google_search_api_key']
    google_search_engine_id = creds['GOOGLE']['google_search_engine_id']
    google_search_endpoint = creds['GOOGLE']['google_search_endpoint']
    # search params
    params = {'q': search_term,
              'key': google_search_api_key, 
              'cx': google_search_engine_id, 
              'count': 50,
              'cr': 'countryBR', 
              'lr': 'lang_pt'}
    # execute query
    try:
        response = requests.get(google_search_endpoint, params=params)
        response.raise_for_status()
        query_raw_results = response.json()
        save_results_file(query_raw_results)
    except Exception as ex:
        raise ex
        
    return response.status_code  

In [11]:
def bing_search(search_term,creds):
    # load credentials
    bing_search_api_key = creds['BING']['bing_search_api_key']
    bing_search_endpoint = creds['BING']['bing_search_endpoint']
    # search params
    headers = {'Ocp-Apim-Subscription-Key': bing_search_api_key}
    params = {'q': search_term,
              # A 2-character country code of the country where the results come from.
              'cc': 'BR',
              # The number of search results to return in the response. 
              # The default is 10 and the maximum value is 50. 
              # he actual number delivered may be less than requested.
              'count': 50,
              # The market where the results come from.
              'mkt': 'pt-BR',
              # A comma-delimited list of answers to include in the response.
              'responseFilter': 'Webpages',
             }
    # execute query
    try:
        response = requests.get(bing_search_endpoint, headers=headers, params=params)
        response.raise_for_status()
        query_raw_results = response.json()
        save_results_file(query_raw_results)
    except Exception as ex:
        raise ex
        
    return response.status_code  

In [68]:
def load_sch(file_sch,results_folder=SEARCH_RESULTS_FOLDER,grace_period=0):
    
    # load SCH database
    usecols = [0,1,11,12,13,14,15]
    dtype = {'Número de Homologação': 'str'}
    parse_dates = [0]
    date_format = '%d/%m/%Y'
    
    df_sch = pd.read_csv(file_sch,sep=';',usecols=usecols,dtype=dtype,parse_dates=parse_dates,date_format=date_format)
    df_sch = df_sch[df_sch['Categoria do Produto']==2]
    df_sch = df_sch.sort_values(by='Data da Homologação',ascending=False)
    df_sch = df_sch.drop_duplicates(subset='Número de Homologação')

    # load search history
    results_files = list(Path(results_folder).glob('*.json'))
    for file in results_files:
        search_date, search_engine, search_term, _ = re.split('[_.]',file.name)
        search_date = datetime.strptime(search_date,RESULT_TS_FORMAT).date()
        search_history.append([search_term,search_date])
    columns = ['Número de Homologação', 'Última Pesquisa']
    df_search_history = pd.DataFrame(search_history,columns=columns)
    df_search_history = df_search_history.sort_values(by=columns,ascending=[True,False])
    df_search_history = df_search_history.drop_duplicates(subset='Número de Homologação')

    # merge both dataframes
    df_sch = df_sch.merge(df_search_history,how='left').fillna(-1)

    # filter products certifieds before grace period
    if grace_period > 0:
        certification_date_limit = datetime.today().date() - timedelta(days=grace_period)
        certification_date_limit = certification_date_limit.strftime('%Y-%m-%d')
        df_sch = df_sch[df_sch['Data da Homologação']<=certification_date_limit]

    return df_sch

# Carga e preparação dos dados

In [69]:
file_sch = 'datasets/produtos_certificados.zip'
df_sch = load_sch(file_sch)

df_sch

,Data da Homologação,Número de Homologação,Nome do Fabricante,Modelo,Nome Comercial,Categoria do Produto,Tipo do Produto,Última Pesquisa
0,2024-05-20,085242314103,"Dongguan At Electronics Technology Co.,LTD.",MD-12259,-1,2,Equipamento de Radiocomunicação de Radiação Re...,-1
1,2024-05-20,170012013522,Acronex S.R.L,Unimap Box V5E,-1,2,Transceptor de Radiação Restrita,-1
2,2024-05-20,060552404444,"Hangzhou Hikvision Digital Technology Co., Ltd.",DS-K2702WX-ECKV,-1,2,Transceptor de Radiação Restrita,-1
3,2024-05-20,039542401925,LG Electronics Inc.,XG2TBK,XBOOM,2,Transceptor de Radiação Restrita,-1
4,2024-05-20,051692401739,"SAMSUNG ELECTRONICS CO., LTD",SM-L300,-1,2,Transceptor de Radiação Restrita,-1
...,...,...,...,...,...,...,...,...
36621,2001-09-28,020210101343,Celwave Division of Radio Frequency Systems Inc.,APX189015,APX189015,2,Antena Direcional,-1
36622,2001-09-28,020200101343,Celwave Division of Radio Frequency Systems Inc.,AP189011,AP189011,2,Antena Direcional,-1
36623,2001-09-28,020170101293,KATHREIN MOBILCOM BRASIL LTDA,749 147/AP13-850/065 6T,749 147/AP13-850/065 6T,2,Antena Direcional,-1
36624,2001-09-28,020190101343,Celwave Division of Radio Frequency Systems Inc.,AP186512,AP186512,2,Antena Direcional,-1


# Pesquisa

Nessa etapa serão feitas as pesquisas no Bing e no Google. Os resultados de cada consulta serão armazenados em um arquivo .json contendo a resposta bruta da consulta. 

Os arquivos serão armazenados na pasta *datasets/searchresults* com o seguinte formato de nome: `{data/hora da consulta}\_{mecanismo de busca}\_{nº do certificado de homologação}.json`, onde:
- data/hora da consulta: data e hora que a consulta foi realizada, no formato `%Y%m%d%H%M%S%f`
- mecanismo de busca: `GOOGLE` ou `BING`
- nº do certificado de homologação: número do certificado de homologação (apenas números)

# Tratamento dos dados

Nessa etapa cada arquivo será lido e dele extraído o nome ou título e o resumo (*snippet*) da resposta para consolidar e criar a wordcloud

In [3]:
file_sch = 'datasets/produtos_certificados.zip'
grace_period = 90

# load SCH database
usecols = [0,1,11,12,13,14,15]
dtype = {'Número de Homologação': 'str'}
parse_dates = [0]
date_format = '%d/%m/%Y'
    
df_sch = pd.read_csv(file_sch,sep=';',usecols=usecols,dtype=dtype,parse_dates=parse_dates,date_format=date_format)
df_sch = df_sch[df_sch['Categoria do Produto']==2]
df_sch = df_sch.sort_values(by='Data da Homologação',ascending=False)
df_sch = df_sch.drop_duplicates(subset='Número de Homologação')

# load search history
results_files = list(Path(results_folder).glob('*.json'))
for file in results_files:
    search_date, search_engine, search_term, _ = re.split('[_.]',file.name)
    search_date = datetime.strptime(search_date,RESULT_TS_FORMAT).date()
    search_history.append([search_term,search_date])
columns = ['Número de Homologação', 'Última Pesquisa']
df_search_history = pd.DataFrame(search_history,columns=columns)
df_search_history = df_search_history.sort_values(by=columns,ascending=[True,False])
df_search_history = df_search_history.drop_duplicates(subset='Número de Homologação')

# merge both dataframes
df_sch = df_sch.merge(df_search_history,how='left').fillna(-1)

NameError: name 'results_folder' is not defined

In [102]:
results_folder = Path(SEARCH_RESULTS_FOLDER)
results_files = list(results_folder.glob('*.json'))

df = pd.DataFrame([parse_result_file(file) for file in results_files])
df.iloc[:,:-1].to_excel('wordcloud.xlsx',index=False)
df

,ID,DataHora,Computador,Usuário,Homologação,Atributo,Valor,Buscadora
0,c5e83e4e-243b-4325-8dfe-fc40c1c07dd7,23/02/2024 15:01:29,ES6927559DTL,E!,03724-22-14637,WordCloud,"{""metaData"": {""Version"": 1, ""Source"": ""GOOGLE""...",GOOGLE
1,5f43bb0c-62ad-4f40-a55f-0c2b761c0a41,23/02/2024 15:01:30,ES6927559DTL,E!,02035-19-01516,WordCloud,"{""metaData"": {""Version"": 1, ""Source"": ""GOOGLE""...",GOOGLE
2,44311a1b-99c7-4465-8490-cc0756d2dc43,23/02/2024 15:01:31,ES6927559DTL,E!,02018-19-01516,WordCloud,"{""metaData"": {""Version"": 1, ""Source"": ""GOOGLE""...",GOOGLE
3,e59c98ab-bd35-434e-86cd-a18a2604aabf,23/02/2024 15:01:31,ES6927559DTL,E!,06618-19-01516,WordCloud,"{""metaData"": {""Version"": 1, ""Source"": ""GOOGLE""...",GOOGLE
4,606f4af2-49d8-483b-afae-12c4b8d9456f,23/02/2024 15:01:32,ES6927559DTL,E!,12303-20-01516,WordCloud,"{""metaData"": {""Version"": 1, ""Source"": ""GOOGLE""...",GOOGLE
...,...,...,...,...,...,...,...,...
2175,df77d3a9-3dab-4866-8c24-613f0b8e5c9b,01/03/2024 12:17:56,ES6927559DTL,E!,00108-15-01699,WordCloud,"{""metaData"": {""Version"": 1, ""Source"": ""BING"", ...",BING
2176,69cd29d3-310c-4d04-b9e7-5f72e9c59f93,01/03/2024 12:17:57,ES6927559DTL,E!,04886-16-01086,WordCloud,"{""metaData"": {""Version"": 1, ""Source"": ""BING"", ...",BING
2177,92877972-d703-46d1-86ec-d94819d9c2a1,01/03/2024 12:17:58,ES6927559DTL,E!,05311-16-01138,WordCloud,"{""metaData"": {""Version"": 1, ""Source"": ""BING"", ...",BING
2178,f948be1a-8d37-4816-844b-5f6be1542caf,22/05/2024 10:27:11,ES6927559DTL,E!,03724-22-14637,WordCloud,"{""metaData"": {""Version"": 1, ""Source"": ""GOOGLE""...",GOOGLE
